# 安装环境

In [ ]:
!pip3 install fschat

In [ ]:
!pip install pyllama -U

# 设置根目录

In [ ]:
# 加载云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir '/content/drive/MyDrive/vicuna'

In [1]:
import os

os.chdir('/content/drive/MyDrive/vicuna')
!pwd
!ls

/content/drive/MyDrive/vicuna
controller.log		 output        vicuna-7b-delta-v1.1
model_worker_3efdd9.log  pyllama_data  vicuna_data


# 获取llama权重

7B大小13G，13B大小25G，fp16

In [ ]:
!python -m llama.download --model_size 7B
# !python -m llama.download --model_size 13B

❤️ Resume download is supported. You can ctrl-c and rerun the program to resume the downloading
✅ pyllama_data/tokenizer.model
✅ pyllama_data/tokenizer_checklist.chk
tokenizer.model: OK
downloading file to pyllama_data/7B/consolidated.00.pth ...please wait for a few minutes ...
✅ pyllama_data/7B/consolidated.00.pth
✅ pyllama_data/7B/params.json
✅ pyllama_data/7B/checklist.chk
Checking checksums
consolidated.00.pth: OK
params.json: OK


In [ ]:
!wget -P ./pyllama_data https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/llama/convert_llama_weights_to_hf.py

--2023-06-03 17:32:35--  https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/convert_llama_weights_to_hf.py
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./pyllama_data/convert_llama_weights_to_hf.py.1’

convert_llama_weigh     [ <=>                ] 269.08K  --.-KB/s    in 0.03s   

2023-06-03 17:32:35 (7.62 MB/s) - ‘./pyllama_data/convert_llama_weights_to_hf.py.1’ saved [275542]



# 生成羊驼模型

In [ ]:
!python3 -m fastchat.model.apply_delta --base ./output/7B --target vicuna_data/vicuna-7b-v1.1 --delta lmsys/vicuna-7b-delta-v1.1 --low-cpu-mem
# !python3 -m fastchat.model.apply_delta --base ./pyllama_data/output/13B --target vicuna_data/vicuna-13b-v1.1 --delta lmsys/vicuna-13b-delta-v1.1

2023-06-03 19:15:08.591570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Split files for the base model to /tmp/tmpxbxtn6vl
100% 2/2 [03:35<00:00, 107.53s/it]
Split files for the delta weights to /tmp/tmphv6l4ic9
Fetching 10 files:   0% 0/10 [00:00<?, ?it/s]

Fetching 10 files: 100% 10/10 [00:00<00:00, 39.41it/s]
100% 2/2 [02:37<00:00, 78.86s/it]
Applying the delta
4it [07:09, 107.45s/it]
Saving the target model to vicuna_data/vicuna-7b-v1.1


# 羊驼模型运行

- 13B需要28G显存，无法直接使用，可以使用--load-8bit



In [ ]:
!python3 -m fastchat.serve.cli --model-path vicuna_data/vicuna-7b-v1.1
# !python3 -m fastchat.serve.cli --model-path vicuna_data/vicuna-13b-v1.1 --load-8bit

2023-06-03 19:44:09.793326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 4/4 [03:54<00:00, 58.55s/it]
USER: 中国的首都在哪
ASSISTANT: 中国的首府是北京。
USER: exit...
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /usr/lib/python3.10/runpy.py:196 in _run_module_as_main                      │
│                                                                              │
│   193 │   main_globals = sys.modules["__main__"].__dict__                    │
│   194 │   if alter_argv:                                                     │
│   195 │   │   sys.argv[0] = mod_spec.origin                                  │
│ ❱ 196 │   return _run_code(code, main_globals, None,                         │
│   197 │   │   │   │   │    "__main__", mod_spec)                             │
│   198                                                                        │
│   199 def run_module(mod_name, init